In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('/content/Train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4866 entries, 0 to 4865
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   A1                  4866 non-null   int64 
 1   A2                  4866 non-null   int64 
 2   A3                  4866 non-null   int64 
 3   A4                  4866 non-null   int64 
 4   A5                  4866 non-null   int64 
 5   A6                  4866 non-null   int64 
 6   A7                  4866 non-null   int64 
 7   A8                  4866 non-null   int64 
 8   A9                  4866 non-null   int64 
 9   A10                 4866 non-null   int64 
 10  Sleep_Disorders     4583 non-null   object
 11  Hearing_Impairment  4571 non-null   object
 12  Vision_Problems     4831 non-null   object
 13  Sex                 4866 non-null   object
 14  Jauundice           4866 non-null   object
 15  Family_ASD          4866 non-null   object
 16  Class               4866

In [2]:
df.isna().sum()

,0
A1,0
A2,0
A3,0
A4,0
A5,0
A6,0
A7,0
A8,0
A9,0
A10,0


In [3]:
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Sleep_Disorders,Hearing_Impairment,Vision_Problems,Sex,Jauundice,Family_ASD,Class
0,1,0,0,0,1,0,1,1,0,0,Yes,No,Yes,f,no,no,NO
1,1,1,0,0,1,1,1,1,1,1,Yes,Yes,No,m,no,no,YES
2,1,1,1,1,1,0,1,1,1,1,No,No,Yes,m,yes,no,YES
3,1,1,1,1,1,0,0,1,1,1,Yes,Yes,Yes,f,no,yes,YES
4,1,1,0,0,0,0,0,1,0,0,No,Yes,No,f,no,yes,NO


In [4]:
for col in df.columns:
  print(df[col].value_counts())
  print()

A1
1    3335
0    1531
Name: count, dtype: int64

A2
0    2480
1    2386
Name: count, dtype: int64

A3
1    2630
0    2236
Name: count, dtype: int64

A4
1    2634
0    2232
Name: count, dtype: int64

A5
1    2582
0    2284
Name: count, dtype: int64

A6
0    2735
1    2131
Name: count, dtype: int64

A7
1    2510
0    2356
Name: count, dtype: int64

A8
1    2539
0    2327
Name: count, dtype: int64

A9
0    2885
1    1981
Name: count, dtype: int64

A10
1    2880
0    1986
Name: count, dtype: int64

Sleep_Disorders
No     2677
Yes    1448
yes     457
NO        1
Name: count, dtype: int64

Hearing_Impairment
No      2455
Yes     1746
NO       368
Yes        1
YES        1
Name: count, dtype: int64

Vision_Problems
Yes    2831
No     1038
NO      962
Name: count, dtype: int64

Sex
m    2941
f    1925
Name: count, dtype: int64

Jauundice
no     3962
yes     904
Name: count, dtype: int64

Family_ASD
no     3957
yes     909
Name: count, dtype: int64

Class
NO     3548
YES    1318
Name: count, d

In [5]:
object_cols = df.select_dtypes(include='object').columns.drop('Sex')
for col in object_cols:
  df[col] =  df[col].str.lower().str.strip()
  df[col] = df[col].map({'no': int(0), 'yes': int(1)})

df['Sex'] =  df['Sex'].str.lower().str.strip()
df['Sex'] = df['Sex'].map({'f': int(0), 'm': int(1)})

In [6]:
for col in df.columns:
    mode_val = df[col].mode(dropna=True)
    if not mode_val.empty:
        df[col] = df[col].fillna(mode_val[0])

float_cols = df.select_dtypes(include=['float']).columns
for col in float_cols:
    df[col] = df[col].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4866 entries, 0 to 4865
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   A1                  4866 non-null   int64
 1   A2                  4866 non-null   int64
 2   A3                  4866 non-null   int64
 3   A4                  4866 non-null   int64
 4   A5                  4866 non-null   int64
 5   A6                  4866 non-null   int64
 6   A7                  4866 non-null   int64
 7   A8                  4866 non-null   int64
 8   A9                  4866 non-null   int64
 9   A10                 4866 non-null   int64
 10  Sleep_Disorders     4866 non-null   int64
 11  Hearing_Impairment  4866 non-null   int64
 12  Vision_Problems     4866 non-null   int64
 13  Sex                 4866 non-null   int64
 14  Jauundice           4866 non-null   int64
 15  Family_ASD          4866 non-null   int64
 16  Class               4866 non-null   int64


In [7]:
for col in object_cols:
  print(df[col].value_counts())
  print()

Sleep_Disorders
0    2961
1    1905
Name: count, dtype: int64

Hearing_Impairment
0    3118
1    1748
Name: count, dtype: int64

Vision_Problems
1    2866
0    2000
Name: count, dtype: int64

Jauundice
0    3962
1     904
Name: count, dtype: int64

Family_ASD
0    3957
1     909
Name: count, dtype: int64

Class
0    3548
1    1318
Name: count, dtype: int64



In [8]:
df.head(15)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Sleep_Disorders,Hearing_Impairment,Vision_Problems,Sex,Jauundice,Family_ASD,Class
0,1,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0
1,1,1,0,0,1,1,1,1,1,1,1,1,0,1,0,0,1
2,1,1,1,1,1,0,1,1,1,1,0,0,1,1,1,0,1
3,1,1,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1
4,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
5,0,0,1,1,0,0,0,0,0,0,1,1,1,1,0,0,0
6,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0
7,1,1,0,0,1,1,0,1,1,0,1,0,0,0,1,0,0
8,1,1,0,1,1,1,0,1,1,0,1,0,0,0,1,1,1
9,1,1,1,1,1,0,0,1,0,1,1,1,1,1,0,0,1


In [9]:
df.isna().sum()

,0
A1,0
A2,0
A3,0
A4,0
A5,0
A6,0
A7,0
A8,0
A9,0
A10,0


In [10]:
X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = make_pipeline(StandardScaler(), LogisticRegression())

# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# scores = cross_val_score(pipeline, X, y, cv=kf, scoring='accuracy')
# print("K-Fold Accuracy Scores:", scores)
# print("Mean Accuracy:", np.mean(scores))

pipeline.fit(X_train, y_train)
y_val_pred = pipeline.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))


Validation Accuracy: 0.9106776180698152


In [11]:
df_test = pd.read_csv('/content/Test.csv')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   A1                  1209 non-null   int64 
 1   A2                  1209 non-null   int64 
 2   A3                  1209 non-null   int64 
 3   A4                  1209 non-null   int64 
 4   A5                  1209 non-null   int64 
 5   A6                  1209 non-null   int64 
 6   A7                  1209 non-null   int64 
 7   A8                  1209 non-null   int64 
 8   A9                  1209 non-null   int64 
 9   A10                 1209 non-null   int64 
 10  Sleep_Disorders     1190 non-null   object
 11  Hearing_Impairment  1209 non-null   object
 12  Vision_Problems     1209 non-null   object
 13  Sex                 1209 non-null   object
 14  Jauundice           1209 non-null   object
 15  Family_ASD          1209 non-null   object
dtypes: int64(10), object(6)


In [12]:
df_test.head(10)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Sleep_Disorders,Hearing_Impairment,Vision_Problems,Sex,Jauundice,Family_ASD
0,1,1,0,1,0,0,1,1,0,0,Yes,No,Yes,f,no,no
1,0,1,1,1,0,1,1,0,1,0,Yes,Yes,No,m,no,no
2,1,1,1,0,1,1,1,1,1,1,No,No,Yes,m,yes,yes
3,1,1,1,1,1,1,1,1,0,0,Yes,Yes,Yes,f,no,no
4,1,1,1,1,1,1,1,1,1,1,No,Yes,No,f,no,no
5,0,1,1,1,0,1,1,0,1,0,Yes,Yes,Yes,m,no,no
6,1,0,0,0,1,0,0,1,0,1,No,No,Yes,f,no,no
7,1,0,0,0,1,0,0,1,0,1,Yes,No,No,f,yes,no
8,1,0,0,0,1,0,0,1,0,1,Yes,No,No,f,yes,no
9,0,1,1,1,0,1,1,0,1,0,Yes,Yes,Yes,m,no,no


In [13]:
df_test.isna().sum()

,0
A1,0
A2,0
A3,0
A4,0
A5,0
A6,0
A7,0
A8,0
A9,0
A10,0


In [14]:
object_cols = df_test.select_dtypes(include='object').columns.drop('Sex')
for col in object_cols:
  df_test[col] =  df_test[col].str.lower().str.strip()
  df_test[col] = df_test[col].map({'no': int(0), 'yes': int(1)})

df_test['Sex'] =  df_test['Sex'].str.lower().str.strip()
df_test['Sex'] = df_test['Sex'].map({'f': int(0), 'm': int(1)})

for col in df_test.columns:
    mode_val = df_test[col].mode(dropna=True)
    if not mode_val.empty:
        df_test[col] = df_test[col].fillna(mode_val[0])

float_cols = df_test.select_dtypes(include=['float']).columns
for col in float_cols:
    df_test[col] = df_test[col].astype(int)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   A1                  1209 non-null   int64
 1   A2                  1209 non-null   int64
 2   A3                  1209 non-null   int64
 3   A4                  1209 non-null   int64
 4   A5                  1209 non-null   int64
 5   A6                  1209 non-null   int64
 6   A7                  1209 non-null   int64
 7   A8                  1209 non-null   int64
 8   A9                  1209 non-null   int64
 9   A10                 1209 non-null   int64
 10  Sleep_Disorders     1209 non-null   int64
 11  Hearing_Impairment  1209 non-null   int64
 12  Vision_Problems     1209 non-null   int64
 13  Sex                 1209 non-null   int64
 14  Jauundice           1209 non-null   int64
 15  Family_ASD          1209 non-null   int64
dtypes: int64(16)
memory usage: 151.3 KB


In [15]:
X_test = df_test

y_test_pred = pipeline.predict(X_test)

submission_from_test = pd.DataFrame({
    'ID': X_test.index + 1,
    'TARGET': y_test_pred
})

submission_from_test.to_csv('submission_from_test.csv', index=False)